In [37]:
import numpy as np
import pandas as pd
import os, glob
import nibabel as nib

In [38]:
# Call subjects from desired institution(this code only works for one institutions)
seg_dir = 'segmentation images directory'
roi_dir = 'Far and Near ROI images directory'

# Sorted all the subjects in same orders
segm_dir = sorted(glob.glob(os.path.join(seg_dir, "*_segm*.nii.gz")))
near_dir = sorted(glob.glob(os.path.join(roi_dir, "*_Infiltrated*.nii.gz")))
far_dir = sorted(glob.glob(os.path.join(roi_dir, "*_Pure*.nii.gz")))

In [39]:
# Load nifti files into numpy array
def LoadingImage(dir):
    
    nifti_image = nib.load(dir)
    image = np.asarray(nifti_image.dataobj)
    header = nifti_image.header
    imgaffine = nifti_image.affine
    
    return image, header, imgaffine

In [40]:
# Separately collects unique subject name from files name
def getFilename(full_dir):
    _,filename = full_dir.split('\\')
    print(filename)
    subject,_,_,_,_= filename.split('_')
    return str(subject)

In [41]:
# Randomly selects the voxels from assinged ROI
def get_random_voxel_coordinates(label, n):
    # Find the coordinates of voxels with value bigger than 0
    one_coords = np.argwhere((label > 0))

    # Grab x,y,z coordinate of the randomly selected voxels.
    one_coords = one_coords[(one_coords[:, 0] >= 2) & (one_coords[:, 0] < label.shape[0] - 2) & 
                            (one_coords[:, 1] >= 2) & (one_coords[:, 1] < label.shape[1] - 2) & 
                            (one_coords[:, 2] >= 2) & (one_coords[:, 2] < label.shape[2] - 2)]

    # Check if there are enough voxels with value 1
    if len(one_coords) < n:
        print(len(one_coords))
        n = len(one_coords)

    # Randomly select n coordinates
    random_indices = np.random.choice(len(one_coords), n, replace=False)
    random_coords = one_coords[random_indices]

    return random_coords

In [ ]:
for i in range(len(segm_dir)):

    # Seg (240, 240, 155)
    seg_img, seg_hdr, seg_imgaffine = LoadingImage(segm_dir[i])
    # call near image (240, 240, 155)
    near_img, near_hdr, near_imgaffine = LoadingImage(near_dir[i])
    # call rec image (240, 240, 155)
    far_img, far_hdr, far_imgaffine = LoadingImage(far_dir[i])

    # select random n numbers of samples
    n = 100
    # apply function of random sampling
    near_random_coords = get_random_voxel_coordinates(near_img, n)

    far_random_coords = get_random_voxel_coordinates(far_img, n)

    # filename
    filename = getFilename(segm_dir[i])

    # Save the randomly selected coordinates to a CSV file
    near_df = pd.DataFrame(near_random_coords, columns=['x', 'y', 'z'])
    near_df.to_csv('save csv file with selected coordinates into ouput directory', index=False)

    # Save the randomly selected coordinates to a CSV file
    far_df = pd.DataFrame(far_random_coords, columns=['x', 'y', 'z'])
    far_df.to_csv('save csv file with selected coordinates into ouput directory', index=False)